In [ ]:
# | default_exp create_vegetation_parameters

In [ ]:
# | hide
from nbdev.showdoc import *
from fastcore.test import *
from fastcore.utils import *

In [ ]:
# | export
import os
import collections
import pandas as pd
import pandera as pa
from typing import Dict
from pathlib import Path
from pandera.typing import Series

In [ ]:
# | export



def create_stand_parameters(





) -> Dict:  # Dictionary containing parameters
    ""

    # Assert parameters ---------------------------------------------------------


    # Create vegetation_parameters from function inputs -------------------------



## Create vegetation parameters 

In [ ]:
#create_vegetation_parameters(
#
#)